In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

print(" Librerías importadas correctamente")

: 

In [ ]:
class DataLoader:
    """Maneja la carga y preparación de datos desde archivos"""
    
    @staticmethod
    def cargar_desde_csv(ruta_archivo):
        """Carga datos desde archivo CSV con formato de fecha/hora"""
        print(" Cargando datos desde archivo...")
        df = pd.read_csv(ruta_archivo)
        print(f"   ✓ {len(df)} registros leídos")
        
        if 'Fecha' in df.columns and 'Hora' in df.columns:
            print(" Procesando fechas y horas...")
            df['timestamp'] = pd.to_datetime(
                df['Fecha'] + ' ' + df['Hora'], 
                format='%d/%m/%y %H:%M'
            )
            df.set_index('timestamp', inplace=True)
            df.rename(columns={'Temp': 'temperatura', 'Hum': 'humedad'}, inplace=True)
            df.drop(['Fecha', 'Hora'], axis=1, inplace=True)
            print("   ✓ Datos preparados correctamente")
        
        return df

In [ ]:
class StatisticalAnalyzer:
    """Calcula estadísticas descriptivas de series temporales"""
    
    @staticmethod
    def calcular_resumen(serie):
        """Calcula resumen estadístico de una serie"""
        stats = serie.describe()
        
        return {
            'Count': int(stats['count']),
            'Mean': round(stats['mean'], 2),
            'Std': round(stats['std'], 2),
            'Min': round(stats['min'], 2),
            '25%': round(stats['25%'], 2),
            '50%': round(stats['50%'], 2),
            '75%': round(stats['75%'], 2),
            'Max': round(stats['max'], 2)
        }
    
    @staticmethod
    def analizar_dataframe(df, columnas):
        """Analiza múltiples columnas de un DataFrame"""
        print(" Calculando estadísticas descriptivas...")
        resultados = {}
        for columna in columnas:
            if columna in df.columns:
                resultados[columna] = StatisticalAnalyzer.calcular_resumen(df[columna])
                print(f"   ✓ {columna.capitalize()}: Media={resultados[columna]['Mean']}, Desv={resultados[columna]['Std']}")
        return resultados

In [ ]:
class ARIMAForecaster:
    
    def __init__(self, orden=(1, 1, 1)):
        self.orden = orden
        self.modelo_fit = None
        self.serie_original = None
    
    def entrenar(self, serie):
        print(f"🤖 Entrenando modelo ARIMA{self.orden}...")
        self.serie_original = serie.values
        modelo = ARIMA(serie, order=self.orden)
        self.modelo_fit = modelo.fit()
        print(f"   ✓ Modelo entrenado (AIC: {self.modelo_fit.aic:.2f})")
        return self
    
    def pronosticar(self, pasos=6):
        if self.modelo_fit is None:
            raise ValueError("Modelo no entrenado. Ejecuta entrenar() primero")
        
        print(f"🔮 Generando pronóstico para {pasos} pasos ({pasos*5} minutos)...")
        pronostico = self.modelo_fit.forecast(steps=pasos)
        print(f"   ✓ Pronóstico generado")
        return pronostico
    
    def obtener_metricas(self):
        if self.modelo_fit is None:
            return None
        
        return {
            'AIC': round(self.modelo_fit.aic, 2),
            'BIC': round(self.modelo_fit.bic, 2),
            'Log-Likelihood': round(self.modelo_fit.llf, 2)
        }
    
    def obtener_parametros(self):
        if self.modelo_fit is None:
            return None
        
        return {nombre: round(valor, 6) 
                for nombre, valor in self.modelo_fit.params.items()}
    
    def obtener_residuos(self):
        if self.modelo_fit is None:
            return None
        
        residuos = self.modelo_fit.resid
        return {
            'media': round(np.mean(residuos), 6),
            'desv_std': round(np.std(residuos), 6),
            'valores': residuos
        }

In [ ]:
class ReportGenerator:
    
    @staticmethod
    def mostrar_resumen_consola(estadisticas):
        print("\n" + "="*70)
        print("RESUMEN ESTADÍSTICO")
        print("="*70)
        
        columnas = list(estadisticas.keys())
        encabezados = ['Métrica'] + columnas
        print(f"{encabezados[0]:<10} {encabezados[1]:>20} {encabezados[2]:>20}")
        print("-"*70)
        
        metricas = estadisticas[columnas[0]].keys()
        for metrica in metricas:
            valores = [str(estadisticas[col][metrica]) for col in columnas]
            print(f"{metrica:<10} {valores[0]:>20} {valores[1]:>20}")
        
        print("="*70)
    
    @staticmethod
    def mostrar_pronostico_detallado(nombre_variable, pronostico, valor_actual, 
                                     orden, parametros, metricas, residuos):
        print("\n" + "="*80)
        print(f"PRONÓSTICO ARIMA{orden} - {nombre_variable.upper()}")
        print("="*80)
        
        print(f"\n[1] CONFIGURACIÓN")
        print(f"p (autorregresivo) = {orden[0]}")
        print(f"d (diferenciación)  = {orden[1]}")
        print(f"q (media móvil)     = {orden[2]}")
        
        print(f"\n[2] PARÁMETROS ESTIMADOS")
        for nombre, valor in parametros.items():
            print(f"  {nombre}: {valor}")
        
        print(f"\n[3] RESIDUOS DEL MODELO")
        print(f"Media: {residuos['media']}")
        print(f"Desv. estándar: {residuos['desv_std']}")
        
        print(f"\n[4] PRONÓSTICO 6 PASOS (30 MINUTOS)")
        print("-" * 80)
        print(f"{'Paso':<8} {'Tiempo':<15} {'Valor':<15} {'Cambio':<15}")
        print("-" * 80)
        
        for i, valor in enumerate(pronostico, 1):
            minutos = i * 5
            cambio = valor - valor_actual
            print(f"{i:<8} t+{minutos:2d} min     {valor:>12.4f}   {cambio:>+12.4f}")
        
        print("-" * 80)
        
        print(f"\n[5] MÉTRICAS DE BONDAD DE AJUSTE")
        for metrica, valor in metricas.items():
            print(f"{metrica}: {valor}")
        
        print(f"\n[6] RESUMEN")
        print(f"Valor actual:         {valor_actual:.2f}")
        print(f"Pronóstico (t+30min): {pronostico[-1]:.2f}")
        print(f"Cambio esperado:      {pronostico[-1] - valor_actual:+.2f}")
        print("="*80)
    
    @staticmethod
    def exportar_a_txt(ruta_archivo, df, estadisticas, pronosticos):
        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("REPORTE DE ANÁLISIS - TERMÓMETRO DE HUMEDAD Y TEMPERATURA\n")
            f.write("="*80 + "\n\n")
            
            f.write(f"Fecha: {pd.Timestamp.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
            f.write(f"Registros: {len(df)}\n")
            f.write(f"Periodo: {df.index.min()} a {df.index.max()}\n\n")
            
            for variable, stats in estadisticas.items():
                f.write("="*80 + "\n")
                f.write(f"RESUMEN ESTADÍSTICO - {variable.upper()}\n")
                f.write("="*80 + "\n")
                for key, value in stats.items():
                    f.write(f"{key:8s}: {value}\n")
                f.write("\n")
            
            for variable, pronostico in pronosticos.items():
                f.write("="*80 + "\n")
                f.write(f"PRONÓSTICO - {variable.upper()} (30 minutos)\n")
                f.write("="*80 + "\n")
                for i, valor in enumerate(pronostico, 1):
                    f.write(f"Paso {i} (t+{i*5} min): {valor:.2f}\n")
                f.write("\n")

In [ ]:
class AnalisisTemperaturaHumedad:
    
    def __init__(self, ruta_archivo):
        self.ruta_archivo = ruta_archivo
        self.df = None
        self.estadisticas = {}
        self.pronosticos = {}
        self.modelos = {}
    
    def cargar_datos(self):
        print("\n" + "="*80)
        print("PASO 1: CARGA DE DATOS")
        print("="*80)
        self.df = DataLoader.cargar_desde_csv(self.ruta_archivo)
        print(f" Periodo: {self.df.index.min()} a {self.df.index.max()}")
        print(f" {len(self.df)} registros cargados correctamente\n")
        return self
    
    def calcular_estadisticas(self, columnas=['temperatura', 'humedad']):
        print("="*80)
        print("PASO 2: ANÁLISIS ESTADÍSTICO")
        print("="*80)
        self.estadisticas = StatisticalAnalyzer.analizar_dataframe(self.df, columnas)
        print(" Estadísticas calculadas\n")
        return self
    
    def pronosticar(self, configuracion=None):
        print("="*80)
        print("PASO 3: GENERACIÓN DE PRONÓSTICOS")
        print("="*80)
        if configuracion is None:
            configuracion = {
                'temperatura': (1, 1, 1),
                'humedad': (1, 1, 1)
            }
        
        for variable, orden in configuracion.items():
            if variable in self.df.columns:
                print(f"\n Variable: {variable.upper()}")
                modelo = ARIMAForecaster(orden)
                modelo.entrenar(self.df[variable])
                self.pronosticos[variable] = modelo.pronosticar(pasos=6)
                self.modelos[variable] = modelo
        
        print("\n Todos los pronósticos generados\n")
        return self
    
    def mostrar_resultados(self, detallado=True):
        print("\n" + "="*80)
        print("ANÁLISIS DE DATOS - TERMÓMETRO CON HUMEDAD")
        print("="*80)
        
        ReportGenerator.mostrar_resumen_consola(self.estadisticas)
        
        if detallado:
            for variable, modelo in self.modelos.items():
                ReportGenerator.mostrar_pronostico_detallado(
                    nombre_variable=variable,
                    pronostico=self.pronosticos[variable],
                    valor_actual=self.df[variable].iloc[-1],
                    orden=modelo.orden,
                    parametros=modelo.obtener_parametros(),
                    metricas=modelo.obtener_metricas(),
                    residuos=modelo.obtener_residuos()
                )
        
        print("\n" + "="*80)
        print("RESUMEN FINAL")
        print("="*80)
        for variable in self.pronosticos.keys():
            actual = self.df[variable].iloc[-1]
            futuro = self.pronosticos[variable][-1]
            cambio = futuro - actual
            unidad = "°C" if variable == "temperatura" else "%"
            print(f"{variable.capitalize()}: {actual:.2f}{unidad} → {futuro:.2f}{unidad} ({cambio:+.2f}{unidad})")
        print("="*80)
        
        return self
    
    def exportar(self, ruta_salida='resultados_analisis.txt'):
        ReportGenerator.exportar_a_txt(
            ruta_salida, 
            self.df, 
            self.estadisticas, 
            self.pronosticos
        )
        print(f"\nResultados exportados: '{ruta_salida}'")
        return self
    
    def ejecutar_analisis_completo(self, exportar=True, detallado=True):
        self.cargar_datos()
        self.calcular_estadisticas()
        self.pronosticar()
        self.mostrar_resultados(detallado=detallado)
        
        if exportar:
            self.exportar()
        
        return {
            'dataframe': self.df,
            'estadisticas': self.estadisticas,
            'pronosticos': self.pronosticos,
            'modelos': self.modelos
        }

In [ ]:
analisis = AnalisisTemperaturaHumedad(r"C:\Users\Alberto Marquez\Desktop\ascii\datos_sd_D.txt")
resultados = analisis.ejecutar_analisis_completo()